In [1]:
from wdtools import *

In [2]:
# this can take about 7 minutes
start = time.time()
all_taxlot = combine_taxlot()
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 7.0 minutes to complete


In [3]:
setID = 'Set003'
nm_to_add = 2143

In [4]:
df = combined_reindexed_data(setID=setID, nm_to_add=nm_to_add) 

In [5]:
# export = False
gdf = match_wd_data_with_taxlot(df=df, setID=setID, all_taxlot=all_taxlot, nm_to_add=nm_to_add, export=False)

In [6]:
unmatched_df = report_unmatched(gdf=gdf, setID=setID, nm_to_add=nm_to_add, mute = False)

it is about 12.38% of data in the original 735 records unmatched
there are 0 records (0.0% of the original records) without parcel id


In [7]:
unmatched_df = unmatched_df.sort_values(by=['wetdet_delin_number'], ascending=False, key=lambda x: x.map(x.value_counts()))

In [119]:
unmatched_df.to_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setID}.csv'), index=False) 

In [8]:
def get_lot_number_from_taxlot(x):
    if 'ROAD' in x:
        res = 'ROW'
    elif 'WATER' in x:
        res = 'Water'
    else:
        res = str(int(x.split('--')[1]))
    return res

In [249]:
wd_id = 'WD2017-0177'
county_to_check = 'Columbia'
trsqq_to_check = '05N01W34CD'
parcel_IDs_to_check = '600,700,800,All part'
lon = -122.798881
lat = 45.868417
taxlot = all_taxlot
year = '2017'

In [254]:
wd_pt  = get_point_from_lonlat(lon = lon, lat = lat)
tID = extract_taxlot_info(wd_pt = wd_pt, taxlot = taxlot, year = year)
trsqq, trsqq_dict, df = read_trsqq()
trsqq_to_check_c = pad_string(trsqq_to_check)
lots_to_check = [x for x in get_lot_numbers(parcel_IDs_to_check)]
trsqq_to_compare = trsqq_dict[tID]
trsqq_to_compare_c = pad_string(trsqq_to_compare)
lots_to_compare = df.loc[df.trsqq==trsqq_to_compare, 'ORTaxlot'].values
lots_to_compare = list(map(get_lot_number_from_taxlot, lots_to_compare))

In [19]:
# this function only works when the coordinates are accurate and one-on-one match among WD ID, trsqq, parcel IDs, and coordinate
# limitation - one WD record (possible with multiple records with different trasqq and parcel IDs) has only one coordindate
def review_wd_record_w_coord(wd_id, county_to_check, trsqq_to_check, parcel_IDs_to_check, lon, lat, taxlot, year):
        print(f'reviewing {wd_id}')
        wd_pt  = get_point_from_lonlat(lon = lon, lat = lat)
        tID = extract_taxlot_info(wd_pt = wd_pt, taxlot = taxlot, year = year)
        trsqq, trsqq_dict, df = read_trsqq()
        trsqq_to_check_c = pad_string(trsqq_to_check)
        lots_to_check = get_lot_numbers(parcel_IDs_to_check)
        trsqq_to_compare = trsqq_dict[tID]
        trsqq_to_compare_c = pad_string(trsqq_to_compare)
        lots_to_compare = df.loc[df.trsqq==trsqq_to_compare, 'ORTaxlot'].values
        lots_to_compare = list(map(get_lot_number_from_taxlot, lots_to_compare))
        if "away" not in tID:
            if trsqq_to_compare_c == trsqq_to_check_c:
                print("trsqq matched, checking county code...")
                cnty_code = int(get_county_code_from_lonlat(lon, lat))
                county_to_compare = [key for key, value in cnt_dict.items() if value == cnty_code]
                # need to check the typos in the county name first
                if county_to_check == county_to_compare[0]:
                    print("county code is corrected, need to check lot numbers...")
                    if any([x not in lots_to_compare for x in lots_to_check]):
                        lots_to_correct = [x for x in lots_to_check if x not in lots_to_compare]
                        cor_type, cor_notes = "lot number", f'lot number {lots_to_correct} might be incorrect'
                        print("lot numbers might be wrong...")
                    else:
                        notes = 'lot numbers seem to be correct, need to review'
                        print(notes)
                        cor_type, cor_notes = None, notes
                else:
                    cor_type, cor_notes = "county", f'from {county_to_check} to {county_to_compare}'
                    print("corrected county...")
            else:
                # check the lot numbers
                lots_matched = [x for x in lots_to_check if x in lots_to_compare]
                if len(lots_matched) > 0:
                    if len(lots_matched) == len(lots_to_check):
                        print("all lots are matched...")
                        cor_type, cor_notes = report_trsqq_correction(trsqq_to_check_c, trsqq_to_compare_c)
                        print("corrected trsqq...")
                    else:
                        notes = f"some lots are not matched, need to review trsqq, the close-match is {trsqq_to_compare}"
                        print(notes)
                        print(f"lots to check: {lots_to_check}, and lots to compare: {lots_to_compare}")
                        cor_type, cor_notes = "to review", notes
                else:
                    notes = f"there is not any matched lot, need to review trsqq, the close-match is {trsqq_to_compare}"
                    print(notes)
                    print(f"lots to check: {lots_to_check}, and lots to compare: {lots_to_compare}")
                    cor_type, cor_notes = "to review", notes
        else:
            cor_type, cor_notes = "coordinate", f'coordinate might be incorrect, nearby taxlot is {tID}'
        return cor_type, cor_notes

In [258]:
review_wd_record_w_coord(wd_id, county_to_check, trsqq_to_check, parcel_IDs_to_check, lon, lat, taxlot, year)

reviewing WD2017-0177
all lots are matched...
corrected trsqq...


('section number and QQ', 'from 34 and CD to 05 and 00')

In [10]:
def report_trsqq_correction(trsqq_to_check, trsqq_to_compare, to_correct=False):
    diff_idx, correct_trsqq_elements, errors = compare_trsqq(trsqq_to_check, trsqq_to_compare)
    if len(diff_idx) == 1:
        keylist = trsqq_correction_dict[diff_idx[0]]
        res = f'from {errors} to {correct_trsqq_elements}'
        if to_correct:
            return trsqq_to_compare
        else:
            return keylist, res
    else:
        keylist = [trsqq_correction_dict.get(key) for key in diff_idx]
        if len(keylist) == 2:
            joined_keys = ' and '.join(keylist)
            joined_errors = ' and '.join(errors)
            joined_corrections = ' and '.join(correct_trsqq_elements)
            res = f'from {joined_errors} to {joined_corrections}'
        else:
            joined_keys = join_list_elements(keylist)
            res = f'from {join_list_elements(errors)} to {join_list_elements(correct_trsqq_elements)}'
        if to_correct:
            return trsqq_to_compare
        else:
            return joined_keys, res

In [11]:
def correct_trsqq(trsqq_to_check, lon, lat, taxlot, year):
    #print(trsqq_to_check)
    wd_pt  = get_point_from_lonlat(lon = lon, lat = lat)
    tID = extract_taxlot_info(wd_pt = wd_pt, taxlot = taxlot, year = year)
    trsqq, trsqq_dict, df = read_trsqq()
    trsqq_to_check_c = pad_string(trsqq_to_check)
    trsqq_to_compare = trsqq_dict[tID]
    trsqq_to_compare_c = pad_string(trsqq_to_compare)
    res = report_trsqq_correction(trsqq_to_check_c, trsqq_to_compare_c, to_correct=True)    
    return res, tID

In [68]:
res

(['section number', 'QQ'],
 'section number and QQ are corrected from 34 and CD to 05 and 00 respectively')

In [12]:
def compare_trsqq(trsqq_to_check, trsqq_to_compare):
    numbers1, letters1 = separate_numbers_letters(trsqq_to_check[:-2])
    numbers2, letters2 = separate_numbers_letters(trsqq_to_compare[:-2])
    letters1.append(trsqq_to_check[-2:])
    letters2.append(trsqq_to_compare[-2:])
    trsqq_to_check_lst = remove_tuple_format(combine_lists(numbers1, letters1))
    trsqq_to_compare_lst = remove_tuple_format(combine_lists(numbers2, letters2))
    diff_idx = find_different_indices(trsqq_to_check_lst, trsqq_to_compare_lst)
    if len(diff_idx) != 0:
        if len(diff_idx) == 1:
            correct_trsqq_elements = trsqq_to_compare_lst[diff_idx[0]]
            errors = trsqq_to_check_lst[diff_idx[0]]
        else:
            correct_trsqq_elements = get_list_elements_by_index(trsqq_to_compare_lst, diff_idx)
            errors = get_list_elements_by_index(trsqq_to_check_lst, diff_idx)
    return diff_idx, correct_trsqq_elements, errors

In [52]:
trsqq_to_check='05N01W34CD'; trsqq_to_compare='05N01W0500'

In [53]:
numbers1, letters1 = separate_numbers_letters(trsqq_to_check[:-2])
numbers2, letters2 = separate_numbers_letters(trsqq_to_compare[:-2])
letters1.append(trsqq_to_check[-2:])
letters2.append(trsqq_to_compare[-2:])
trsqq_to_check_lst = remove_tuple_format(combine_lists(numbers1, letters1))
trsqq_to_compare_lst = remove_tuple_format(combine_lists(numbers2, letters2))
diff_idx = find_different_indices(trsqq_to_check_lst, trsqq_to_compare_lst)

In [13]:
unmatched = pd.read_excel(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setID}_r2.xlsx'))

In [121]:
unmatched.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot'],
      dtype='object')

In [122]:
unmatched.shape

(11, 25)

In [14]:
to_review = unmatched[((unmatched.latitude.astype(str) != 'nan')|(unmatched.longitude.astype(str) != 'nan')) & (unmatched.missinglot == 'N')]

In [128]:
to_review.shape

(10, 25)

In [15]:
outdf = to_review[['wetdet_delin_number', 'trsqq', 'parcel_id', 'county', 'latitude', 'longitude', 'DecisionLink', 'record_ID', 'IDyear']]

In [17]:
outdf.parcel_id.values

array(['3000p', '106p', 8009001000, '6000p', 5602, 1100, 201, '300p', 300,
       '600,700,800,All part'], dtype=object)

In [22]:
outdf.loc[:,'correct_type'], outdf.loc[:,'correction'] = zip(*outdf.apply(lambda row: review_wd_record_w_coord(wd_id = row.wetdet_delin_number, 
                                                                        county_to_check = row.county, 
                                                                        trsqq_to_check = row.trsqq, 
                                                                        parcel_IDs_to_check = row.parcel_id, 
                                                                        lon = row.longitude, 
                                                                        lat = row.latitude, 
                                                                        taxlot = all_taxlot, 
                                                                        year = row.IDyear), axis = 1))

reviewing WD2019-0512
all lots are matched...
corrected trsqq...
reviewing WD2021-0192
there is not any matched lot, need to review trsqq, the close-match is 06S11W34AD
lots to check: ['106'], and lots to compare: ['100', '200', '300', '400', '500', '600', '700', '800', '900', '1000', '1100', '1200', '1300', '1400', '1500', '1600', '1700', '1800', '1900', '2000', '2100', '2200', '2300', '2400', '2500', '2600', '2700', '2800', '2900', '3000', '3100', '3200', '3300', '3400', '3500', '3700', '3800', '3900', '4000', '4100', '4200', '4300', '4400', '4401', '4500', '4600', '4700', '4800', '4900', '5000', '5100', '5200', '5300', '5400', '5500', '5501', 'ROW', 'ROW']
reviewing WD2017-0267
all lots are matched...
corrected trsqq...
reviewing WD2020-0265
all lots are matched...
corrected trsqq...
reviewing WD2020-0261
all lots are matched...
corrected trsqq...
reviewing WD2017-0214
all lots are matched...
corrected trsqq...
reviewing WD2018-0242
there is not any matched lot, need to review trsqq

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [23]:
sel = ~outdf.correct_type.isin(['county', 'lot number', 'coordinate'])

In [24]:
outdf.loc[sel, 'cor_trsqq'], outdf.loc[sel, 'ORTaxlot'] = zip(*outdf.loc[sel,:].apply(lambda row: correct_trsqq(trsqq_to_check = row.trsqq, 
                                                                               lon = row.longitude,
                                                                               lat = row.latitude,
                                                                               taxlot = all_taxlot,
                                                                               year = row.IDyear), axis = 1))

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [25]:
outdf.to_excel(os.path.join(inpath + '\\output\\to_review\\', f'review_unmatched_{setID}.xlsx'), index=False)

In [136]:
outdf.IDyear.values

array([2019, 2021, 2017, 2020, 2020, 2017, 2018, 2022, 2021, 2017],
      dtype=int64)

In [151]:
all_taxlot.columns

Index(['year', 'ORTaxlot', 'geometry'], dtype='object')

In [190]:
outdf.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'county', 'latitude',
       'longitude', 'DecisionLink', 'record_ID', 'IDyear', 'correct_type',
       'correction', 'cor_trsqq', 'ORTaxlot'],
      dtype='object')

In [26]:
outdf['pairs'] = list(zip(outdf['IDyear'].astype(str), outdf['ORTaxlot']))

<ipython-input-26-2c82acb7f74c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outdf['pairs'] = list(zip(outdf['IDyear'].astype(str), outdf['ORTaxlot']))


In [27]:
taxlots_to_review = all_taxlot[all_taxlot[['year', 'ORTaxlot']].apply(tuple, axis=1).isin(outdf.pairs.values)]

In [28]:
taxlots_to_review_2 = taxlots_to_review.merge(outdf, on='ORTaxlot')

In [31]:
taxlots_to_review_2.drop(columns=['pairs'], inplace=True)

In [32]:
taxlots_to_review_2.to_file(os.path.join(inpath + '\\output\\to_review\\', f'review_unmatched_{setID}_r2.shp'))

<ipython-input-32-2f2d8f405b7b>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  taxlots_to_review_2.to_file(os.path.join(inpath + '\\output\\to_review\\', f'review_unmatched_{setID}_r2.shp'))
